In [1]:
# Built by Kayne and given to me on March 11, 2021

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta, date, datetime, date
import os
from sklearn.model_selection import train_test_split
import datarobot as dr

# Pre-processing

#### Ingest data, remove SIZE_ACT and re-format dates

In [2]:
df = pd.read_csv('transaction_data_h.csv',header=0)
df["date_purchase_request"] = pd.to_datetime(df['date_purchase_request'],format='%Y-%m-%d',errors='coerce')
df = df.drop_duplicates()
df = df.groupby(df.columns.to_list()[:-1]).sum().reset_index()
df.head()
#df.shape

,cust_id,date_purchase_request,purchase_no,purchase_item_no,line_number,option_number,size_act,brand,sub_category,category,category_group,size,response
0,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,bottoms,16,0
1,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,bottoms,18,1
2,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,bottoms,20,0
3,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,bottoms,22,0
4,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,bottoms,24,0


In [3]:
custIds = df["cust_id"].drop_duplicates()
sizes = df["size"].drop_duplicates() 
items = df["line_number"].drop_duplicates()
len(custIds)

366556

#### Add on new features on from previous purchase info

In [4]:
allCombos = df[["cust_id","date_purchase_request","line_number","size"]][df["response"] == 1].drop_duplicates()
grouped =  allCombos.groupby(['cust_id'])
allCombos['Rank'] = grouped['date_purchase_request'].transform(lambda x: pd.factorize(x, sort=True)[0]+1)
allCombos['key'] = 0

cross = pd.merge(allCombos[["cust_id","key","Rank"]], allCombos[["cust_id","key","Rank"]],on=["cust_id",'key']).rename(columns={"Rank_x":"Rank","Rank_y":"Prev_Rank"})
cross = cross[(cross["Rank"] > cross["Prev_Rank"]) & (cross["Rank"] >= 1) & (cross["Prev_Rank"] >= 1)].drop("key",axis=1).drop_duplicates()

final = cross.merge(allCombos[["cust_id","Rank","date_purchase_request"]].drop_duplicates(),on=["cust_id","Rank"],how="left")
final = final.merge(allCombos[["cust_id","Rank","size","line_number","date_purchase_request"]].rename(columns={"Rank":"Prev_Rank","size":"prev_size","date_purchase_request":"prev_date_purchase_request","line_number":"prev_line_number"}).drop_duplicates(),on=["cust_id","Prev_Rank"],how="left")
final['Trans_Rank'] = final.groupby(["cust_id","Rank"])["prev_date_purchase_request"].rank(method='first',ascending=False)
final['Trans_Rank_C'] = "L"+final['Trans_Rank'].astype(str)
final.head()

,cust_id,Rank,Prev_Rank,date_purchase_request,prev_size,prev_line_number,prev_date_purchase_request,Trans_Rank,Trans_Rank_C
0,0001ca92bcd0a45aed94b61338c2bb0d,2,1,2019-08-28,18,NHLM3,2019-04-30,1.0,L1.0
1,0001ca92bcd0a45aed94b61338c2bb0d,2,1,2019-08-28,18,NHLKY,2019-04-30,2.0,L2.0
2,000226ed2106ff2769f2cf210ec5e2ac,2,1,2019-09-28,XS,LTQTE,2017-12-18,1.0,L1.0
3,000277e1669671b024277cf1fc1ee7a9,2,1,2018-10-20,10,MKALP,2018-10-17,1.0,L1.0
4,000277e1669671b024277cf1fc1ee7a9,3,1,2019-02-26,10,MKALP,2018-10-17,2.0,L2.0


#### Add on new features for the previous 5 purchases

In [5]:
pivot1 = final[final["Trans_Rank"] <= 50]
pivot1 = pd.pivot_table(pivot1, index=['cust_id',"Rank","date_purchase_request"],columns='Trans_Rank_C',values=["prev_size","prev_line_number"],aggfunc='first',fill_value="N/A")
pivot1.reset_index(inplace=True)
pivot1.columns = ["_".join(x) for x in pivot1.columns.ravel()]
pivot1 = pivot1.rename(columns={"cust_id_":"cust_id","date_purchase_request_":"date_purchase_request"}).drop(columns=["Rank_"],axis=1)
pivot1["last_5_line_number"] = (pivot1["prev_line_number_L1.0"].astype(str)+" "+pivot1["prev_line_number_L2.0"].astype(str)+" "+pivot1["prev_line_number_L3.0"].astype(str)+" "+pivot1["prev_line_number_L4.0"].astype(str)+" "+pivot1["prev_line_number_L5.0"].astype(str)).str.replace('N/A','')
pivot1["last_5_size"] = (pivot1["prev_size_L1.0"].astype(str)+" "+pivot1["prev_size_L2.0"].astype(str)+" "+pivot1["prev_size_L3.0"].astype(str)+" "+pivot1["prev_size_L4.0"].astype(str)+" "+pivot1["prev_size_L5.0"].astype(str)).replace({'N/A':''}).str.replace('N/A','')
pivot1.head()

,cust_id,date_purchase_request,prev_line_number_L1.0,prev_line_number_L10.0,prev_line_number_L11.0,prev_line_number_L12.0,prev_line_number_L13.0,prev_line_number_L14.0,prev_line_number_L15.0,prev_line_number_L16.0,...,prev_size_L48.0,prev_size_L49.0,prev_size_L5.0,prev_size_L50.0,prev_size_L6.0,prev_size_L7.0,prev_size_L8.0,prev_size_L9.0,last_5_line_number,last_5_size
0,0001ca92bcd0a45aed94b61338c2bb0d,2019-08-28,NHLM3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,NHLM3 NHLKY,18 18
1,000226ed2106ff2769f2cf210ec5e2ac,2019-09-28,LTQTE,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,LTQTE,XS
2,000277e1669671b024277cf1fc1ee7a9,2018-10-20,MKALP,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,MKALP,10
3,000277e1669671b024277cf1fc1ee7a9,2019-02-26,MMV3Y,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,MMV3Y MKALP,10 10
4,00029190767aa56d9b3bd825a972ebcd,2018-11-06,MR3YG,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,MR3YG,8


In [6]:
df = df.merge(pivot1,on=["cust_id","date_purchase_request"],how="left")

#### Add on features for every size very bought

In [7]:
pivot2 = final.groupby(["cust_id","Rank","date_purchase_request","prev_size"]).size().reset_index(name='counts')
pivot2 = pd.pivot_table(pivot2, index=['cust_id',"Rank","date_purchase_request"],columns='prev_size',values="counts",fill_value=0)
pivot2 = pivot2[pivot2.columns.to_list()[3:]].add_prefix('size_')
pivot2.reset_index(inplace=True)
pivot2.head()

prev_size,cust_id,Rank,date_purchase_request,size_10 D/DD,size_10 E/F,size_10-12,size_10=28,size_10=M,size_10=MREGULAR,size_10=S,...,size_XS=6,size_XXL,size_XXL/18,size_XXL=16,size_XXL=16REGULAR,size_XXLD/E,size_XXS,size_XXS/6,size_XXXL,size_XXXXL
0,0001ca92bcd0a45aed94b61338c2bb0d,2,2019-08-28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000226ed2106ff2769f2cf210ec5e2ac,2,2019-09-28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000277e1669671b024277cf1fc1ee7a9,2,2018-10-20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000277e1669671b024277cf1fc1ee7a9,3,2019-02-26,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,00029190767aa56d9b3bd825a972ebcd,2,2018-11-06,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = df.merge(pivot2,on=["cust_id","date_purchase_request"],how="left")\
            .drop(["size_act","option_number","purchase_item_no","purchase_no","date_purchase_request"],axis=1)\
            .fillna(0,inplace=True)

In [15]:
df.head()
#df.shape

,cust_id,date_purchase_request,purchase_no,purchase_item_no,line_number,option_number,size_act,brand,sub_category,category,...,size_XS=6,size_XXL,size_XXL/18,size_XXL=16,size_XXL=16REGULAR,size_XXLD/E,size_XXS,size_XXS/6,size_XXXL,size_XXXXL
0,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000419c5c8eebdb9c91c827a18ea4f,2019-05-09,1,1,MTAMG,MTKKP,18,V by Very Curve,womens_jeans,jeans,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Clean up dataset

In [ ]:
df2 = df.drop(["size_act","option_number","purchase_item_no","purchase_no","date_purchase_request"],axis=1)
df2.head()

#### Split into train and score

In [ ]:
#train, score = np.split(df.sample(frac=1), [int(.7*len(df))])

In [7]:
allCombos = df[["cust_id","line_number"]].drop_duplicates()
allCombos.head()

train, score = np.split(allCombos.sample(frac=1), [int(.7*len(allCombos))])

train = df.merge(train,on=["cust_id","line_number"],how="inner")
score = df.merge(score,on=["cust_id","line_number"],how="inner")

#### Have a look at distribution

In [156]:
train.response.value_counts()
#train.head()

0    9166698
1    1204970
Name: response, dtype: int64

In [157]:
score.response.value_counts()
#score.head()

0    3926495
1     516410
Name: response, dtype: int64

#### Export the datasets

In [8]:
train.to_csv("transaction_data_h_train.csv.gz",index=False,compression='gzip')
score.to_csv("transaction_data_h_score.csv.gz",index=False,compression='gzip')

# DataRobot model training

#### Connect to DR and run Autopilot

In [9]:
dr.Client(config_path='drconfig.yaml')

In [10]:
#primary_dataset = dr.Dataset.create_from_in_memory_data(train)
primary_dataset = dr.Dataset.create_from_file(file_path="transaction_data_h_train.csv.gz")

In [162]:
#proj = dr.Project.get("5f11e3c39e69e50184d8b861")

In [11]:
proj = dr.Project.create_from_dataset(primary_dataset.id,project_name='TVG_SizeRecc_withL50Purchases')

In [12]:
#proj = dr.Project.create(train,project_name='TVG_SizeRecc',max_wait=3600

In [15]:
holdout = 8
validation = 8
kFolds = 5
#CVDetails = dr.StratifiedCV(holdout_pct=holdout,reps=kFolds)
CVDetails = dr.StratifiedTVH(holdout_pct=holdout, validation_pct=validation,)
advancedOptions = dr.AdvancedOptions(smart_downsampled=False)

smidgin = 0.05
#cutoff = (100-holdout)*((kFolds-1)/kFolds)+smidgin
cutoff = (100-holdout-validation)+smidgin
cutoff

84.05

In [16]:
proj.set_target(target='response',mode=dr.enums.AUTOPILOT_MODE.FULL_AUTO,worker_count=-1,partitioning_method=CVDetails,advanced_options=advancedOptions,max_wait=3600)

Project(TVG_SizeRecc_withL50Purchases)

#### Find the deepCTR models and run those as well

In [17]:
proj.get_blueprints()

[Blueprint(eXtreme Gradient Boosted Trees Classifier with Early Stopping (learning rate =0.43) (Fast Feature Binning)),
 Blueprint(Elastic-Net Classifier with Naive Bayes Feature Weighting (L2)),
 Blueprint(TensorFlow Deep Learning Classifier),
 Blueprint(eXtreme Gradient Boosted Trees Classifier with Early Stopping (learning rate =0.01) (Fast Feature Binning)),
 Blueprint(Gradient Boosted Greedy Trees Classifier with Early Stopping),
 Blueprint(eXtreme Gradient Boosted Trees Classifier with Early Stopping (learning rate =0.01) (Fast Feature Binning)),
 Blueprint(Light Gradient Boosting on ElasticNet Predictions ),
 Blueprint(Nystroem Kernel SVM Classifier - Forest (5x)),
 Blueprint(Gradient Boosted Trees Classifier with Early Stopping),
 Blueprint(Keras Residual AutoInt Classifier using Training Schedule (3 Attention Layers with 2 Heads, 2 Layers: 100, 100 Units)),
 Blueprint(Keras Deep Residual Neural Network Classifier using Training Schedule (2 Layers: 512, 512 Units)),
 Blueprint(

In [18]:
menuKeras = [m for m in proj.get_blueprints() if m.model_type.startswith('Keras Residual')]
menuKeras

[Blueprint(Keras Residual AutoInt Classifier using Training Schedule (3 Attention Layers with 2 Heads, 2 Layers: 100, 100 Units)),
 Blueprint(Keras Residual Cross Network Classifier using Training Schedule (3 Cross Layers, 4 Layers: 100, 100, 100, 100 Units)),
 Blueprint(Keras Residual Neural Factorization Machine Classifier using Training Schedule (2 Layers: 100, 100 Units))]

In [19]:
NFM = menuKeras[2]  #my location for the "Keras Residual Neural Factorization Machine Classifier"
XNtwk = menuKeras[1]  #my location for the "Keras Residual Cross Network Classifier"
autoInt = menuKeras[0]  #my location for the "Keras Residual AutoInt Classifier"

In [20]:
runNFM = proj.train(NFM)
runXNtwk = proj.train(XNtwk)
runAutoInt = proj.train(autoInt)

In [174]:
dr.models.modeljob.wait_for_async_model_creation(project_id = proj.id,model_job_id = runNFM,max_wait = 3600)
dr.models.modeljob.wait_for_async_model_creation(project_id = proj.id,model_job_id = runXNtwk,max_wait = 3600)
dr.models.modeljob.wait_for_async_model_creation(project_id = proj.id,model_job_id = runAutoInt,max_wait = 3600)

Model('Keras Residual AutoInt Classifier using Training Schedule (3 Attention Layers with 2 Heads, 2 Layers: 100, 100 Units)')

#### Set up a hyperparameter tuning job

In [175]:
models = proj.get_models()

[Model('Keras Residual AutoInt Classifier using Training Schedule (3 Attention Layers with 2 Heads, 2 Layers: 100, 100 Units)'),
 Model('Keras Residual Cross Network Classifier using Training Schedule (3 Cross Layers, 4 Layers: 100, 100, 100, 100 Units)'),
 Model('Keras Residual Neural Factorization Machine Classifier using Training Schedule (2 Layers: 100, 100 Units)'),
 Model('Light Gradient Boosted Trees Classifier with Early Stopping'),
 Model('Light Gradient Boosting on ElasticNet Predictions '),
 Model('Light Gradient Boosting on ElasticNet Predictions '),
 Model('Light Gradient Boosted Trees Classifier with Early Stopping'),
 Model('Decision Tree Classifier (Gini)'),
 Model('Balanced RandomForest Classifier (Entropy)'),
 Model('eXtreme Gradient Boosted Trees Classifier with Early Stopping (learning rate =0.43) (Fast Feature Binning)'),
 Model('Decision Tree Classifier (Gini)'),
 Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)'),
 

##### One-off

In [50]:
model = [m for m in models if ((m.model_type.startswith('Keras Residual AutoInt Classifier')) & (m.sample_pct < cutoff))][0]
model

Model('Keras Residual AutoInt Classifier using Training Schedule (3 Attention Layers with 2 Heads, 2 Layers: 100, 100 Units)')

In [71]:
tune = model.start_advanced_tuning_session()

In [72]:
tune.get_task_names()

['Keras Residual Cross Network Classifier using Training Schedule (3 Cross Layers, 4 Layers: 100, 100, 100, 100 Units)']

In [73]:
task_name = 'Keras Residual Cross Network Classifier using Training Schedule (3 Cross Layers, 4 Layers: 100, 100, 100, 100 Units)'
tune.get_parameter_names(task_name)

['batch_size',
 'cross_network_hidden_bias_initializer',
 'cross_network_hidden_initializer',
 'cross_network_layers',
 'double_batch_size',
 'dropout_type',
 'early_stopping',
 'epochs',
 'hidden_activation',
 'hidden_batch_norm',
 'hidden_bias_initializer',
 'hidden_dropout',
 'hidden_initializer',
 'hidden_l1',
 'hidden_l2',
 'hidden_units',
 'hidden_use_bias',
 'learning_rate',
 'loss',
 'loss_quantile_level',
 'max_batch_size',
 'max_embedding_size',
 'optimizer',
 'output_activation',
 'output_batch_norm',
 'output_bias_initializer',
 'output_initializer',
 'output_l1',
 'output_l2',
 'pass_through_inputs',
 'prediction_batch_size',
 'random_seed',
 'stochastic_weight_average_epochs',
 'training_schedule_curve',
 'training_schedule_cycle_count',
 'training_schedule_cycle_scale',
 'training_schedule_cycle_warm_up_fraction',
 'training_schedule_post_cycle_scale',
 'training_schedule_warm_down_fraction',
 'use_training_schedule']

In [ ]:
tune.set_parameter(task_name=task_name, parameter_name='max_embedding_size', value=32)
tune.set_parameter(task_name=task_name, parameter_name='learning_rate', value=0.02)

In [ ]:
job = tune.run()
new_model = job.get_result_when_complete()

##### Script to loop

In [176]:
autoIntModel = [m for m in models if ((m.model_type.startswith('Keras Residual AutoInt Classifier')) & (m.sample_pct < cutoff))][0]
NFMModel = [m for m in models if ((m.model_type.startswith('Keras Residual Neural Factorization Machine Classifier')) & (m.sample_pct < cutoff))][0]
XNtwkModel = [m for m in models if ((m.model_type.startswith('Keras Residual Cross Network Classifier')) & (m.sample_pct < cutoff))][0]

In [177]:
def keras_tuning(model):
    
    model_params = model.get_advanced_tuning_parameters()['tuning_parameters']
    max_embedding_size = list(filter(lambda param: param['parameter_name'] == 'max_embedding_size', model_params))[0]['default_value']
    batch_size = list(filter(lambda param: param['parameter_name'] == 'batch_size', model_params))[0]['default_value']

    #All DeepCTR blueprints
    max_embedding_sizes = list(map(int,[max_embedding_size,max_embedding_size*2]))
    batch_sizes = list(map(int,[batch_size/2,batch_size,batch_size*2]))

    #CrossNet only
    cross_network_layers = [1,2,3]

    #Neural factorization only
    biout_dropout = [0.3,0.7]

    #AutoInt only
    attention_heads = [1,2]
    attention_layers = [1,2,3]

    #Loop through all relevant tuning parameter combinations
    for i in max_embedding_sizes:
        for j in batch_sizes:
            tune = model.start_advanced_tuning_session()
            task_name = tune.get_task_names()[0]
            tune.get_parameter_names(task_name)
            tune.set_parameter(task_name=task_name, parameter_name='max_embedding_size', value=i)
            tune.set_parameter(task_name=task_name, parameter_name='batch_size', value=j)
            if task_name.startswith('Keras Residual Cross Network Classifier'):
                for k in cross_network_layers:
                    tune.set_parameter(task_name=task_name, parameter_name='cross_network_layers', value=k)
                    try:
                        job = tune.run()
                    except:
                        pass
            if task_name.startswith('Keras Residual AutoInt Classifier'):
                for l in attention_heads:
                    for m in attention_layers:
                        tune.set_parameter(task_name=task_name, parameter_name='attention_heads', value=l)
                        tune.set_parameter(task_name=task_name, parameter_name='attention_layers', value=m)
                        tune.set_parameter(task_name=task_name, parameter_name='attention_embedding_size', value=i)
                        try:
                            job = tune.run()
                        except:
                            pass
            if task_name.startswith('Keras Residual Neural Factorization Machine Classifier'):
                for o in biout_dropout:
                    tune.set_parameter(task_name=task_name, parameter_name='biout_dropout', value=o)
                    try:
                        job = tune.run()
                    except:
                        pass

In [178]:
keras_tuning(model=autoIntModel)
keras_tuning(model=NFMModel)
keras_tuning(model=XNtwkModel)

#### After optimizing, re-train on 100%

In [ ]:
proj.unlock_holdout()

In [74]:
runBestOn100Pct = proj.get_models()[0].train(sample_pct=100)

# Scoring & looking at accuracy of best model

#### Find best model

In [52]:
model = dr.ModelRecommendation.get(proj.id, dr.enums.RECOMMENDED_MODEL_TYPE.RECOMMENDED_FOR_DEPLOYMENT).get_model()
model

AttributeError: 'NoneType' object has no attribute 'get_model'

In [199]:
model = [m for m in models if ((m.model_type.startswith('Keras Residual AutoInt Classifier')) & (m.sample_pct < cutoff))][0]
#model = [m for m in models if ((m.model_type.startswith('Keras Residual Cross Network Classifier')) & (m.sample_pct < cutoff))][0]
#model = [m for m in models if ((m.model_type.startswith('Keras Residual Neural Factorization Machine Classifier')) & (m.sample_pct < cutoff))][0]
model

Model('Keras Residual AutoInt Classifier using Training Schedule (3 Attention Layers with 2 Heads, 2 Layers: 100, 100 Units)')

#### Look at the threshold - for this use case you might want this lower to maximise recall/sensitivity, i.e less False Negatives
#### You may also want to play around with the Profit Curves in the GUI application to optimize choice of threshold

#### Upload scoring dataset and perform scoring

In [200]:
dataset = proj.upload_dataset(score,max_wait=3600) #send code so you only do it once
pred_job = model.request_predictions(dataset.id)
preds = pred_job.get_result_when_complete(max_wait=3600)
predsAll = pd.concat([score, preds], axis=1)
predsAll.head()

,cust_id,line_number,brand,sub_category,category,category_group,size,response,prev_line_number_L1.0,prev_line_number_L2.0,...,prev_size_L2.0,prev_size_L3.0,prev_size_L4.0,prev_size_L5.0,prediction_threshold,prediction,row_id,positive_probability,class_0.0,class_1.0
0,00002fa30f5df73cef97102d1e2890ac,NJUHF,Girls On Film Curve,womens_dresses,dresses,dresses,18,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,0,0.111004,0.888996,0.111004
1,00002fa30f5df73cef97102d1e2890ac,NJUHF,Girls On Film Curve,womens_dresses,dresses,dresses,20,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,1,0.194423,0.805577,0.194423
2,00002fa30f5df73cef97102d1e2890ac,NJUHF,Girls On Film Curve,womens_dresses,dresses,dresses,22,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,2,0.251833,0.748167,0.251833
3,00002fa30f5df73cef97102d1e2890ac,NJUHF,Girls On Film Curve,womens_dresses,dresses,dresses,24,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,3,0.070352,0.929648,0.070352
4,00002fa30f5df73cef97102d1e2890ac,NJUHF,Girls On Film Curve,womens_dresses,dresses,dresses,26,1,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,4,0.037421,0.962579,0.037421


#### Clean up dataset and aggregate up so it is one row per CUST_ID x ITEM_NUMBER

In [201]:
scoredf = predsAll[(predsAll["cust_id"].notnull()) & (predsAll["line_number"].notnull()) & (predsAll["response"].notnull()) & (predsAll["prediction"].notnull())]
scoredf = scoredf.sort_values(by=["cust_id","line_number","positive_probability"],ascending=[False,False,False])
scoredf = scoredf.groupby(["cust_id","line_number"]).first().reset_index()
scoredf.head()

,cust_id,line_number,brand,sub_category,category,category_group,size,response,prev_line_number_L1.0,prev_line_number_L2.0,...,prev_size_L2.0,prev_size_L3.0,prev_size_L4.0,prev_size_L5.0,prediction_threshold,prediction,row_id,positive_probability,class_0.0,class_1.0
0,00002fa30f5df73cef97102d1e2890ac,NJUHF,Girls On Film Curve,womens_dresses,dresses,dresses,22,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,2,0.251833,0.748167,0.251833
1,0000adadd13c7c655c9e08ef17d22446,N6J7W,Monsoon,womens_dresses,dresses,dresses,14,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,7,0.325359,0.674641,0.325359
2,000190e87f24c447cd20056f7a5f022d,LWPHK,adidas Originals,womens_everyday_sports_tights,trousers,bottoms,10,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,13,0.290460,0.709540,0.290460
3,0001ca92bcd0a45aed94b61338c2bb0d,NR7NH,V by Very Curve,womens_skirts,skirts,bottoms,18,1,NHLM3,NHLKY,...,18,NaN,NaN,NaN,0.5,1.0,24,0.541131,0.458869,0.541131
4,0001ff128d96a4af204bf62e81ac342b,NUCP4,V by Very,womens_basics_leggings,trousers,bottoms,12,0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.5,0.0,31,0.194114,0.805886,0.194114


#### Generate accuracy metric

In [202]:
# AutoInt = 48%, XNtwk = x%, NFM = x%
print("Total accuracy = "+str(sum(scoredf["response"])/len(scoredf["response"])))

Total accuracy = 0.49521883458454624


In [ ]:
predsAll[(predsAll["cust_id"] == "00000419c5c8eebdb9c91c827a18ea4f") & (predsAll["line_number"] == "MXLVF")]\
                .sort_values(by=["cust_id","line_number","positive_probability"],ascending=[False,False,False])

In [67]:
predsAll[(predsAll["cust_id"] == "00000419c5c8eebdb9c91c827a18ea4f")] # & (predsAll["category_group"] == "lingerie")]
df[(df["cust_id"] == "00000419c5c8eebdb9c91c827a18ea4f")]

,cust_id,line_number,brand,sub_category,category,category_group,size,response
0,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,16,0
1,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,18,1
2,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,20,0
3,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,22,0
4,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,24,0
5,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,26,0
6,00000419c5c8eebdb9c91c827a18ea4f,MTAMG,V by Very Curve,womens_jeans,jeans,bottoms,28,0
7,00000419c5c8eebdb9c91c827a18ea4f,MXLVF,Pour Moi,womens_lingerie_bras_matching_sets,lingerie,lingerie,40D,0
8,00000419c5c8eebdb9c91c827a18ea4f,MXLVF,Pour Moi,womens_lingerie_bras_matching_sets,lingerie,lingerie,40DD,0
9,00000419c5c8eebdb9c91c827a18ea4f,MXLVF,Pour Moi,womens_lingerie_bras_matching_sets,lingerie,lingerie,40E,0
